In [52]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import hashlib

In [22]:
application_columns = ['Id', 'EventID', 'Computer', 'TimeCreated', 'AccountName', 
           'SubjectDomainName', 'LogonID', 'Shortcut', 'ProcessId', 
           'ProcessName', 'WorkstationName', 'IpAddress', 'CommandLine']
application = pd.read_csv('C:/Users/sarab/Documents/TFM/data/ApplicationUsage.csv', header=None, sep=";")
application.columns = application_columns
application.TimeCreated = pd.to_datetime(application.TimeCreated,infer_datetime_format=True)
application.Id = application.Id.astype(str)
application.EventID = application.EventID.astype('category')
application.Computer = application.Computer.astype(str)
application.AccountName = application.AccountName.astype(str)
application.SubjectDomainName = application.SubjectDomainName.astype(str)
application.LogonID = application.LogonID.astype('category')
application.Shortcut = application.Shortcut.astype(str)

In [23]:
application.head()

,Id,EventID,Computer,TimeCreated,AccountName,SubjectDomainName,LogonID,Shortcut,ProcessId,ProcessName,WorkstationName,IpAddress,CommandLine
0,202109031242153807080,23,VD006.dcloud.local,2021-09-03 14:42:15,adm.aosuna,dcloud,1.0,c:\windows\system32\winlogon.exe,0x00,escritorio redes,NaN,NaN,NaN
1,202109061627137932236,21,VD018.dcloud.local,2021-09-06 18:27:13,administrator,vd018,2.0,c:\windows\system32\winlogon.exe,0x00,vd018,NaN,127.0.0.1,NaN
2,202109061710594887921,24,VD018.dcloud.local,2021-09-06 19:10:59,administrator,vd018,2.0,c:\windows\system32\winlogon.exe,0x00,vd018,NaN,127.0.0.1,NaN
3,202109061854568868705,25,VD018.dcloud.local,2021-09-06 20:54:56,administrator,vd018,2.0,c:\windows\system32\winlogon.exe,0x00,vd018,NaN,127.0.0.1,NaN
4,202109062054141777969,23,VD018.dcloud.local,2021-09-06 22:54:14,administrator,vd018,2.0,c:\windows\system32\winlogon.exe,0x00,vd018,NaN,NaN,NaN


In [79]:
user_columns = ['TimeCreated', 'UserName', 'DomainName', 'ServerName', 'SessionID', 'ClientName',
                'ClientIpAddress', 'DeviceType', 'ConnectionStatus', 'ConnectorBandwidth', 'ConnectorLatency', 'ConnectorType',
                'ConnectorVersion', 'WebsiteName', 'OperatingSystem', 'ServicePack', 'HostName']
users = pd.read_csv('C:/Users/sarab/Documents/TFM/data/UsserSession.csv', sep=';', header=None)
users.columns = user_columns
users.TimeCreated = pd.to_datetime(users.TimeCreated,infer_datetime_format=True)
users.UserName = users.UserName.astype(str)
def get_enrolment_year(x):
    enrolment_year_list = x.split(".")
    if len(enrolment_year_list)==3:
        try:
            return int(enrolment_year_list[2])
        except:
            return np.NaN
    else:
        return np.NaN
users['enrolment_year'] = users.UserName.apply(lambda x: get_enrolment_year(x))
users.UserName = users.UserName.apply(lambda x: hashlib.sha256(x.encode("utf-8")).hexdigest())
users.DomainName = users.DomainName.astype(str)
users.DomainName = users.DomainName.str.lower()
users.ServerName = users.ServerName.astype(str)
users.ServerName = users.ServerName.str.lower()
def get_campus(x):
    server_name_list = x.split("-")
    if len(server_name_list)==4:
        return server_name_list[0]
    else:
        return np.NaN
users['Campus'] = users.ServerName.apply(lambda x: get_campus(x))
def get_lab(x):
    server_name_list = x.split("-")
    if len(server_name_list)==4:
        return server_name_list[1]
    else:
        return np.NaN
users['Lab'] = users.ServerName.apply(lambda x: get_lab(x))
def get_aula(x):
    server_name_list = x.split("-")
    if len(server_name_list)==4:
        return server_name_list[2]
    else:
        return np.NaN
users['Aula'] = users.ServerName.apply(lambda x: get_aula(x))
def get_computer(x):
    server_name_list = x.split("-")
    if len(server_name_list)==4:
        return server_name_list[3].split(".")[0]
    else:
        return np.NaN
users['Computer'] = users.ServerName.apply(lambda x: get_computer(x))
users.SessionID = users.SessionID.astype('category')
users.DeviceType = users.DeviceType.astype('category')
users.ConnectionStatus = users.ConnectionStatus.astype('category')
users.ConnectorBandwidth = users.ConnectorBandwidth.astype(int)
users.ConnectorLatency = users.ConnectorLatency.astype(int)
users.ConnectorType = users.ConnectorType.astype('category')
users.drop(['WebsiteName', 'HostName'], inplace=True, axis=1)
users.OperatingSystem = users.OperatingSystem.astype('category')
users.ServicePack = users.ServicePack.astype('category')
users.enrolment_year = users.enrolment_year.astype('double')
users.Computer = users.Computer.astype('double')

In [83]:
users.enrolment_year.unique()

array([2021., 2019., 2020.,   nan, 2018., 2017., 2016., 2022.])

In [128]:
reservations_columns = ['Id', 'Date', 'Cod_Plan', 'Tipo', 'Fecha_Inicio', 'Hora_Inicio', 'Fecha_Fin', 'Hora_Fin', 
                        'Day', 'Campus', 'Edificio', 'Aula', 'Capacidad', 'Software']
reservations = pd.read_csv('C:/Users/sarab/Documents/TFM/data/FLStatsReservaations.csv', sep=';', header=None)
reservations.columns = reservations_columns
reservations = reservations.set_index('Id')
reservations.Date = pd.to_datetime(reservations.Date,infer_datetime_format=True)
reservations.Cod_Plan = reservations.Cod_Plan.astype(int)
reservations.Tipo = reservations.Tipo.astype('category')
reservations.Fecha_Inicio = reservations.Fecha_Inicio + '-' + reservations.Hora_Inicio
reservations.Fecha_Fin = reservations.Fecha_Fin + '-' + reservations.Hora_Fin
reservations.Fecha_Inicio = pd.to_datetime(reservations.Fecha_Inicio,infer_datetime_format=True)
reservations.Fecha_Fin = pd.to_datetime(reservations.Fecha_Fin,infer_datetime_format=True)
reservations.drop(['Hora_Inicio', 'Hora_Fin'], axis=1, inplace=True)
reservations['Year'] = reservations['Fecha_Inicio'].dt.year
reservations['Month'] = reservations['Fecha_Inicio'].dt.month
reservations['Dow'] = reservations['Fecha_Inicio'].dt.dayofweek
reservations['Day'] = reservations['Fecha_Inicio'].dt.day
reservations['Hour_Ini'] = reservations['Fecha_Inicio'].dt.hour
reservations['Hour_Fin'] = reservations['Fecha_Fin'].dt.hour
reservations['Duration'] = reservations.Hour_Fin - reservations.Hour_Ini
reservations.Campus = reservations.Campus.astype('category')
reservations.Edificio = reservations.Edificio.astype(str)
reservations.Aula = reservations.Aula.astype(str)
def cast_capacidad(x):
    try:
        return int(x)
    except:
        emp_lis = []
        for z in x.split(' '):
            if z.isdigit():
                emp_lis.append(int(z))
        return np.sum(emp_lis)
reservations.Capacidad = reservations.Capacidad.apply(lambda x: cast_capacidad(x))
reservations.Software = reservations.Software.astype('category')

In [130]:
reservations.head()

,Date,Cod_Plan,Tipo,Fecha_Inicio,Fecha_Fin,Day,Campus,Edificio,Aula,Capacidad,Software,Year,Month,Dow,Hour_Ini,Hour_Fin,Duration
Id,,,,,,,,,,,,,,,,,
2520a7f7-fef1-4893-ae9e-59fbded9a8bc,2022-02-27 05:30:05.730,2000,PUNTUAL,2021-10-25 15:00:00,2021-10-25 18:00:00,25,ALCORCON,Aulario II,Aula Informática 102,40,Google Chrome,2021,10,0,15,18,3
3f45c2a5-163a-49a1-9ac7-525d8c6bd0b9,2022-02-27 05:30:05.730,2000,PUNTUAL,2021-10-25 15:00:00,2021-10-25 18:00:00,25,ALCORCON,Aulario II,Aula Informática 102,40,Microsoft Office 2010,2021,10,0,15,18,3
bc4d3fcb-1108-4c43-b588-c95ffe22aa9a,2022-02-27 05:30:05.730,2000,PUNTUAL,2021-11-08 15:00:00,2021-11-08 18:00:00,8,ALCORCON,Aulario II,Aula Informática 102,40,Google Chrome,2021,11,0,15,18,3
40dda022-ac5e-43c3-ad63-98a80d8e98b3,2022-02-27 05:30:05.730,2000,PUNTUAL,2021-11-08 15:00:00,2021-11-08 18:00:00,8,ALCORCON,Aulario II,Aula Informática 102,40,Microsoft Office 2010,2021,11,0,15,18,3
5794d899-5af2-4726-93ff-4bb6103b2a3a,2022-02-27 05:30:05.730,2000,PUNTUAL,2021-11-09 12:00:00,2021-11-09 14:00:00,9,ALCORCON,Aulario II,Aula Informática 102,40,Google Chrome,2021,11,1,12,14,2
